# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)
# %reload_ext autoreload


Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [3]:
#call the model creation funtion
import Modelcreation
model = Modelcreation.createmodel()

# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [4]:
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_24 (ZeroPadding2 (None, 102, 102, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_24[0][0]          
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [5]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

In [6]:
# Set layer weights of the model
for name in weights:
    if model.get_layer(name) != None:
        model.get_layer(name).set_weights(weights_dict[name])
    elif model.get_layer(name) != None:
        model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [7]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [8]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_nam      cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()e, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [9]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = ima 'img' is not definedge_to_embedding(image_file, model)
    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    cv2.namedWindow("Face Recognizer")
    #vc = cv2.VideoCapture(0)
    vc=cv2.VideoCapture('/home/vishnu/Desktop/video1.mp4')
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    while vc.isOpened():
        _, rframe  = vc.read()
        frame = cv2.resize(rframe, (640, 480)) 
        img =frame
        height, width, channels = frame.shape
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_ 'img' is not definedimage, input_embeddings, model)
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)
        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [17]:
import sqlite3
import cv2
import numpy as np

cam = cv2.VideoCapture(0)
#cam = cv2.VideoCapture('http://172.16.2.167:8080/video') #taking the image through the webcam

try:
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    def insertOrUpdate(Id,Name,Regno):
        conn = sqlite3.connect("Container.db")
        cmd = "SELECT * FROM Identity WHERE ID="+str(Id)
        cursor = conn.execute(cmd)
        isRecordExist = 0
        for row in cursor:
            isRecordExist=1 
        if(isRecordExist==1):
            cmd = "UPDATE Identity SET Name=' "+str(Name)+" ' WHERE ID="+str(Id)
            cmd = "UPDATE Identity SET Regno=' "+str(Regno)+" ' WHERE ID="+str(Id)
        else:
            cmd ="INSERT INTO Identity(ID,Name,Regno) Values("+str(Id)+",' "+str(Name)+" ',' "+str(Regno)+" ')"  
        conn.execute(cmd)
        conn.commit()
        conn.close()

    Id = input("enter your id : ")
    name = input("Enter Your Name : ")
    reg =input("Enter Your Registration Number : ")

    insertOrUpdate(Id, name,reg)


    count = 0
    while(True):
        ret, frame = cam.read()
        img = cv2.resize(frame, (640, 480)) 
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #general Intuition to detect the faces.

        faces = face_detector.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
            count += 1
            # Save the captured image into the datasets folder
            #cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
            cv2.imwrite("images/" +  name  +"." +str(Id )+'.'+ str(count) + ".jpg", img[y1:y2,x1:x2])
            cv2.imshow('image+window', img)
            cv2.namedWindow("image_window", cv2.WINDOW_NORMAL)
        k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video to terminate the webcam.
        if k == 27:
            break
        elif count >= 10: # Take 30 face sample and stop video
             break
    cam.release()
    cv2.destroyAllWindows()
    
except Exception as e:
    #to destroy the camera
    print('The Error is : ',str(e))
    cam.release()
    cv2.destroyAllWindows() 

enter your id : 12
Enter Your Name : priyansh
Enter Your Registration Number : 457


# Lets run the face recognizer program :-)

In [20]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

Euclidean distance from priyansh.12.1 is 1.2273353
Euclidean distance from none.1.1 is 0.93733865
Euclidean distance from priyansh.12.1 is 0.869877
Euclidean distance from none.1.1 is 1.024006
Euclidean distance from priyansh.12.1 is 1.1691378
Euclidean distance from none.1.1 is 1.2063586
Euclidean distance from priyansh.12.1 is 1.1369828
Euclidean distance from none.1.1 is 1.0331635
Euclidean distance from priyansh.12.1 is 0.9929765
Euclidean distance from none.1.1 is 0.93774146
Euclidean distance from priyansh.12.1 is 1.0980546
Euclidean distance from none.1.1 is 0.9336938
Euclidean distance from priyansh.12.1 is 0.87793183
Euclidean distance from none.1.1 is 0.9887166
Euclidean distance from priyansh.12.1 is 0.867958
Euclidean distance from none.1.1 is 1.0510964
Euclidean distance from priyansh.12.1 is 1.2429368
Euclidean distance from none.1.1 is 0.9364335
Euclidean distance from priyansh.12.1 is 1.2328023
Euclidean distance from none.1.1 is 0.865603
Euclidean distance from priyans

Euclidean distance from priyansh.12.1 is 0.6994298
Euclidean distance from none.1.1 is 1.0697744
Euclidean distance from priyansh.12.1 is 0.75861365
Euclidean distance from none.1.1 is 1.0801263
Euclidean distance from priyansh.12.1 is 0.77210504
Euclidean distance from none.1.1 is 0.9577399
Euclidean distance from priyansh.12.1 is 0.87534773
Euclidean distance from none.1.1 is 0.8981014
Euclidean distance from priyansh.12.1 is 0.9600687
Euclidean distance from none.1.1 is 0.81396526
Euclidean distance from priyansh.12.1 is 0.8392991
Euclidean distance from none.1.1 is 0.9068199
Euclidean distance from priyansh.12.1 is 0.78259116
Euclidean distance from none.1.1 is 1.0304602
Euclidean distance from priyansh.12.1 is 0.89365566
Euclidean distance from none.1.1 is 1.0089434
Euclidean distance from priyansh.12.1 is 0.89346457
Euclidean distance from none.1.1 is 1.0273873
Euclidean distance from priyansh.12.1 is 0.99427545
Euclidean distance from none.1.1 is 1.0255468
Euclidean distance fro

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
